In [406]:
import requests
import pandas as pd
import json
import numpy as np
import math
import re

In [407]:
# get the json from the API and turn it into a dataframe
response = requests.get("http://app-sellaci.eu-central-1.elasticbeanstalk.com/")
data = response.json()
df = pd.json_normalize(data)

In [408]:
df_t = df.T

with open("original.json", "w") as write_file:
    json.dump(df.to_dict(), write_file, indent=4)

In [409]:
# remove nan values
df_t.dropna(axis=0, inplace=True)

In [410]:
# remove isNotConformReason if it is conform
for row in df_t:
    if not df_t[row].isNotConform:
        df_t[row].drop(columns=['isNotConformReason'], inplace=True)


In [411]:
# format phone number
for row in df_t:
    new_number = df_t[row].patientPhoneNumber.replace('iso2', '')
    df_t[row].patientPhoneNumber = '+' + re.sub("[^0-9]", "", new_number)


In [412]:
# remove "unknown" and empty values
df = df_t.T
empty = []

for row in df:
    for idx in df[row].keys():
        if (df_t[idx][row] == 'unknown' or df_t[idx][row] == ''):# or (type(df_t[idx][row]) == list and len(df_t[idx][row]) == 0)):
            empty.append(row)

df.drop(columns=empty, inplace=True)
df_t = df.T

In [413]:
# turn details into list
for row in df_t:
    df_t[row].details = df_t[row].details.split(', ')


In [414]:
# remove empty strings from paymentIntents
new_dict = df_t[1]['cart.paymentIntents'][0].copy()

for key in df_t[1]['cart.paymentIntents'][0].keys():
    if (df_t[1]['cart.paymentIntents'][0][key] == '' or (type(df_t[1]['cart.paymentIntents'][0][key]) == list and len(df_t[1]['cart.paymentIntents'][0][key]) == 0)): 
        del new_dict[key]

df_t[1]['cart.paymentIntents'][0] = new_dict

In [415]:
# write cleaned dataframe to file
with open("result_cleaned.json", "w") as write_file:
    json.dump(df.to_dict(), write_file, indent=4)

In [416]:
# write cleaned transposed dataframe to file
with open("result_transposed_cleaned.json", "w") as write_file:
    json.dump(df_t.to_dict(), write_file, indent=4)

In [417]:
df.describe()

,_id,affiliation,areCotationsAuthorized,canChangeBeneficiary,createdAt,customerCreditHasBeenChecked,customerCreditHasBeenDecreased,details,doctorHasReviewed,doctorId,...,cart.usePaymentIntent,estimatedLength.unit,estimatedLength.utc,estimatedLength.value,initiator.role,snapshot.beneficiary.hasSocialSecurityNumber,snapshot.beneficiary.medicalInsurance.securiteSocialNumber,snapshot.consultation.isPartnershipOfferIncluded,snapshot.doctor.isCME,snapshot.doctor.offer
count,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
unique,2,1,2,1,2,1,1,2,1,2,...,1,2,1,2,1,1,2,1,1,2
top,1202019029,qare,False,True,2020-09-27T21:30:00.116Z,True,False,"[Malade: mal de dos, fièvre, vomissements, cov...",False,102901290192091,...,True,minutes,+0000,10,patient,True,23232323232311,False,False,connect
freq,1,2,1,2,1,2,2,1,2,1,...,2,1,2,1,2,2,1,2,2,1


In [418]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 68 columns):
 #   Column                                                      Non-Null Count  Dtype 
---  ------                                                      --------------  ----- 
 0   _id                                                         2 non-null      object
 1   affiliation                                                 2 non-null      object
 2   areCotationsAuthorized                                      2 non-null      object
 3   canChangeBeneficiary                                        2 non-null      object
 4   createdAt                                                   2 non-null      object
 5   customerCreditHasBeenChecked                                2 non-null      object
 6   customerCreditHasBeenDecreased                              2 non-null      object
 7   details                                                     2 non-null      object
 8   doctorHasRevie

In [419]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)

                  _id affiliation areCotationsAuthorized canChangeBeneficiary  \
0          1202019029        qare                  False                 True   
1  232323231323212098        qare                   True                 True   

                  createdAt customerCreditHasBeenChecked  \
0  2020-09-27T21:30:00.116Z                         True   
1  2021-09-27T21:24:00.049Z                         True   

  customerCreditHasBeenDecreased  \
0                          False   
1                          False   

                                             details doctorHasReviewed  \
0  [Malade: mal de dos, fièvre, vomissements, cov...             False   
1    [Trouble du sommeil, tension, douleurs au dos ]             False   

                 doctorId doctorReminderSent doctorValidation  \
0         102901290192091              False            False   
1  2308863920973287839723              False            False   

         estimatedStartDate                even